In [1]:
# importing packages 
import pandas as pd
!pip install thefuzz[levenshtein]

import thefuzz

from thefuzz import fuzz
from thefuzz import process

In [3]:
# naming what columns you want to use from the excel sheet that is about to be loaded in, change file path
columns_to_use = ['Plant Name', 'Plant State', 'Technology', 'Net Summer Capacity (MW)', 'County', 'Plant ID', 'Status']
eia_path = "C:\\Users\\63141\\Desktop\\github_files_v2\\eia_may_24.xlsx"
df_eia = pd.read_excel(eia_path, sheet_name = "Operating", usecols = columns_to_use, skiprows = 2)

columns_to_use1 = ['Plant Name', 'State', 'Capacity Type - Sub Type', 'Capacity Modeled', 'County', 'EIA Plant Code', 'On-Line Date']
ipm_path = "C:\\Users\\63141\\Desktop\\github_files_v2\\IPM2023.xlsx"
df_ipm = pd.read_excel(ipm_path,sheet_name="qry", usecols=columns_to_use1, skiprows = 1)

In [4]:
# renaming columns to match each other
df_ipm.rename(columns={'EIA Plant Code': 'Plant Code', 'Capacity Type - Sub Type' : 'Technology'}, inplace=True)
df_eia.rename(columns={'Plant ID': 'Plant Code', 'Plant State' : 'State'}, inplace=True)

# loading in the excel with the technology dictionary for the matching, change path
tech_dict_path = "C:\\Users\\63141\\Desktop\\github files\\tech-lookups.xlsx"

tech_dict_eia = pd.read_excel(tech_dict_path, sheet_name = "EIA")
tech_dict_ipm = pd.read_excel(tech_dict_path, sheet_name = "IPM")


In [5]:
# Merge EIA and IPM with tech dictionary to replace technology names 
tech_eia = pd.merge(df_eia, tech_dict_eia, how='left', on='Technology')
tech_ipm = pd.merge(df_ipm, tech_dict_ipm, how='left', on='Technology')

In [6]:
# Go from unit level to plant level by adding together generators with the same plant code, capacity type, and  plant name
tech_eia['Net Summer Capacity (MW)'] = pd.to_numeric(tech_eia['Net Summer Capacity (MW)'], errors='coerce')

grouped_eia = tech_eia.groupby(['Plant Code', 'Merged Capacity Types', 'Plant Name']).agg({
    'Net Summer Capacity (MW)': 'sum',
    "County": "first",
    "State": "first",
    "Status": "first"
}).reset_index()

tech_ipm['Capacity Modeled'] = pd.to_numeric(tech_ipm['Capacity Modeled'], errors='coerce')


grouped_ipm = tech_ipm.groupby(['Plant Code', 'Merged Capacity Types', 'Plant Name']).agg({
    'Capacity Modeled': 'sum',
    "County": "first",
    "State": "first",
    "On-Line Date": "first"
}).reset_index()

In [7]:
df = pd.merge(grouped_eia, grouped_ipm, how='left', on=['Plant Code', 'Merged Capacity Types'], indicator=True)

In [15]:
# creates a new data frame called both that is a filter to includes only plants from IPM/EIA that had merge based on plant code
both = df[df['_merge'].str.contains('both', case=False, na=False)]
#both_path = "C:\\Users\\63141\\Desktop\\github_files_v2\\both.xlsx"
#both.to_excel(both_path, index = False)


In [9]:
grouped_ipm.loc[:, 'key'] = grouped_ipm['Merged Capacity Types'].astype(str) + grouped_ipm['Plant Name'].astype(str)
grouped_eia.loc[:, 'key'] = grouped_eia['Merged Capacity Types'].astype(str) + grouped_eia['Plant Name'].astype(str)
both.loc[:, 'ipm_key'] = both['Merged Capacity Types'].astype(str) + both['Plant Name_y'].astype(str)
both.loc[:, 'eia_key'] = both['Merged Capacity Types'].astype(str) + both['Plant Name_x'].astype(str)


C:\Users\63141\AppData\Local\Temp\ipykernel_3676\2308489190.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  both.loc[:, 'ipm_key'] = both['Merged Capacity Types'].astype(str) + both['Plant Name_y'].astype(str)
C:\Users\63141\AppData\Local\Temp\ipykernel_3676\2308489190.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  both.loc[:, 'eia_key'] = both['Merged Capacity Types'].astype(str) + both['Plant Name_x'].astype(str)


In [10]:
filtered_grouped_ipm = grouped_ipm[~grouped_ipm['key'].isin(both['ipm_key'])]

filtered_grouped_eia = grouped_eia[~grouped_eia['key'].isin(both['eia_key'])]

In [14]:
filtered_grouped_ipm.shape

(2787, 8)

In [12]:
# here I am saving the trimmed down data frames to an excel files because there were problems I encountered when trying to work with the same notebook using the trimmed data frames 
# we will use these export excel files to go to a new notebook 
save_path = "C:\\Users\\63141\\Desktop\\github_files_v2\\filtered_grouped_ipm.xlsx"
filtered_grouped_ipm.to_excel(save_path, index = False)

save_path_eia = "C:\\Users\\63141\\Desktop\\github_files_v2\\filtered_grouped_eia.xlsx"
grouped_eia.to_excel(save_path_eia, index = False)